In [1]:
import glob
import pandas as pd
import subprocess
import pathlib
import ray
import joblib

## prepare dms hotspot table

In [2]:
_dir_1 = '/home/qzeng/project/aging/230907-recall-dmr/Merge_DMR/BothGender.CellType.Age/DMLtest'
_dir_2 = '/home/qzeng/project/aging/230907-recall-dmr/Merge_DMR/2-BothGender.CellType.Age/DMLtest'
filtered_dmr_bed_dir = '/home/qzeng/project/aging/230907-recall-dmr/Merge_DMR/mC_FILTER_DMR_BED'

In [3]:
all_dml_files = list(glob.glob(f'{_dir_1}/*/*.DMLtest.bed')) + list(glob.glob(f'{_dir_2}/*/*.DMLtest.bed'))

In [4]:
use_cts = joblib.load('/home/qzeng/project/aging/metadata/mc_use_cts')
use_cts = [ct.replace(' ','_').replace('/','') for ct in use_cts]

In [5]:
use_dml_files = []
for path in all_dml_files:
    ct = pathlib.Path(path).name.split('.')[0]
    if ct in use_cts:
        use_dml_files.append(path)

In [6]:
import ray
ray.init(_temp_dir = "/tmp/ray_qzeng")

@ray.remote
def process_ct_type(path, _type):
    # Read the CSV file and filter it in one step
    ct = pathlib.Path(path).name.split('.')[0]
    tmpdf = pd.read_csv(path, sep = '\t')
    #tmpdf = pd.read_csv(f"{_dir}/{ct}/{ct}.DMLtest.bed", sep='\t')
    tmpdf = tmpdf[tmpdf['pvals'] < 0.01][['chr', 'pos']]
    tmpdf['end'] = tmpdf['pos']
    tmpdf = tmpdf.dropna()

    # Save the filtered data to a file
    output_file = f'pvalue_filtered_dms_bed/{ct}.bed'
    tmpdf.to_csv(output_file, sep='\t', header=None, index=None)

    # Run the bedtools intersect command
    command = f"bedtools intersect -b {filtered_dmr_bed_dir}/{ct}.{_type}.aDMR.bed -a {output_file} -wa > mC_FILTERED_DMS_BED/{ct}.{_type}.aDMS.bed"
    subprocess.run(command, shell=True)

    # Add a 'length' column with a constant value
    tmpdf = pd.read_csv(f'mC_FILTERED_DMS_BED/{ct}.{_type}.aDMS.bed', header=None, sep='\t')
    tmpdf['length'] = 1
    tmpdf.to_csv(f'mC_FILTERED_DMS_BED/{ct}.{_type}.aDMS.bed', sep='\t', header=None, index=None)

#all_cts = [pathlib.Path(path).name.split('.')[0] for path in use_dml_files[:2]]
all_types = ['Hypo', 'Hyper']

futures = []
for path in use_dml_files:
    for _type in all_types:
        futures.append(process_ct_type.remote(path, _type))

ray.get(futures)


2024-01-18 15:51:53,058	INFO services.py:1476 -- View the Ray dashboard at http://127.0.0.1:8265
(process_ct_type pid=29690) Error: line number 648015 of file pvalue_filtered_dms_bed/PB_Evx2_Glut.bed has 2 fields, but 3 were expected.


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [7]:
subprocess.run("rm pvalue_filtered_dms_bed -rf", shell = True)

CompletedProcess(args='rm pvalue_filtered_dms_bed -rf', returncode=0)

## map bed

In [2]:
dms_dir = "mC_FILTERED_DMS_BED"

In [3]:
pathlib.Path('500_dms_hotspot').mkdir(exist_ok = True)
for _type in ['Hypo','Hyper']:
    all_paths = glob.glob(f"{dms_dir}/*.{_type}.aDMS.bed")
    
    for file in all_paths:
        mt = pathlib.Path(file).name.split('.')[0]
        sort_ = f"sort -k1,1 -k2,2n -o {file} {file}"
        subprocess.run(sort_, shell =True)
        command = f'bedtools map -a mm10_500.bed -b {file} -null 0 -c 4 > 500_dms_hotspot/{mt}.{_type}.map.bed'
        subprocess.run(command , shell = True)

## group command

In [4]:
for _type in ['Hypo','Hyper']:
    all_files = glob.glob(f'500_dms_hotspot/*.{_type}.map.bed')
    command = f'cat {" ".join(all_files)} > 500_result/{_type}.DMS.concat.bed'
    subprocess.run(command, shell = True)
    
    subprocess.run(f"sort -k1,1 -k2,2n -k3,3n -o 500_result/{_type}.DMS.concat.bed 500_result/{_type}.DMS.concat.bed", shell = True)
    
    command = f"bedtools groupby -i 500_result/{_type}.DMS.concat.bed -g 1,2,3 -c 6 -o sum > 500_result/{_type}.DMS.concat.sum.bed"
    subprocess.run(command, shell = True)
    subprocess.run(f"rm 500_result/{_type}.DMS.concat.bed", shell = True)